# Machine Learning Final Project

Justin May, Joe Shenouda, Jonathan Hong

In [64]:
"""
Import Statements
"""
# Python Specific 
import pickle 
import time
import sys
import math

# Data Science 
import numpy as np
import pandas as pd
from sklearn import preprocessing as sklpp
from sklearn import decomposition as skldecomp
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
import csv
import warnings
import re # for regular expressions

# NLP Libraries -- for twitter data preprocessing
import nltk
from nltk.corpus import stopwords # A set of the most common stop words 
from nltk.stem import WordNetLemmatizer # A function to stem words 
from nltk.tokenize import word_tokenize # An auto-tokenizer 
# ---- RUN THESE IF RUNNING FOR THE FIRST TIME ----
#nltk.download('stopwords')
#nltk.download('wordnet')

# Options
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
"""
Helper Function to See Progress 
"""
def progress(count, total, suffix=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', suffix))
    sys.stdout.flush()  # As suggested by Rom Ruben

--------------------------------------------------------------------------------------------------------------------------------------------------

# Images - Classification

## Preprocessing Step

In [4]:
"""
Extracting data into a file
Data are all in the same domain, no need to normalize
"""
data = None
labels = []

file = "images/data_batch_1"
with open(file, 'rb') as fo:
    print("extracting file "+file+"...")
    dict = pickle.load(fo, encoding='bytes')
    temp_data = dict[b'data']
    try:
        data = np.concatenate((data, temp_data), axis=0)
    except:
        data = temp_data
    labels = labels + dict[b'labels']
labels = np.array(labels)
labels = labels.reshape(-1,1)
print("Finished Extracting Features")
print(np.shape(data))

extracting file images/data_batch_1...
Finished Extracting Features
(10000, 3072)


## Preprocessing and Feature Learning For Images Dataset
For preprocessing we centered our dataset to 0 mean and then for feature learning we apply PCA to reduce our dimensions from 3072 to a smaller number of features than contains 95% of the variance of our data.

In [5]:
# First we create a StandardScaler object to 0 mean the data matrix but preserve the variance
stand_scaler = sklpp.StandardScaler(with_mean = True, with_std = False)
# Fits the data matrix to the StandardScaler object defined ^
centered_ImageData = stand_scaler.fit_transform(data)

In [6]:
# Creates a PCA object that reduces the dimensions of our data matrix keeping 95% of the variance
pca_obj = skldecomp.PCA(n_components = 0.95, svd_solver = 'auto')
dim_reducedImageData = pca_obj.fit_transform(centered_ImageData)
np.save('dim_reducedImageData.npy', dim_reducedImageData)

In [16]:
print('Data has been reduced to {} features after PCA'.format(dim_reducedImageData.shape[1]))

Data has been reduced to 209 features after PCA


## (Model 1) Quadratic Discriminant Analysis: Justin

I implemeneted the algorithim we learned in class: 
 - first learn the parameters (expected average, prior probabilities, covariance matricies) 
 - define the discriminant functions to find the best labels 

In [4]:
"""
Pull pca data from saved
"""
data = np.load('dim_reducedImageData.npy')
data = np.concatenate((data,labels),axis=1)
number_of_parameters = len(data[0]) - 1 #209
number_of_total_samples = len(data)
"""
K-Fold cross validation 
"""     
final_estimates = []
k = 10
for i in range(1,k+1):
    training_data = np.concatenate((data[0:(i-1)*int(number_of_total_samples/k)],data[i*int(number_of_total_samples/k):number_of_total_samples]),axis=0)
    testing_data = data[(i-1)*int(number_of_total_samples/k):i*int(number_of_total_samples/k)]
    number_of_training_samples = len(training_data)
    """
    Define Parameters 
    """
    mu = [[0 for j in range(number_of_parameters)] for i in range(10)] # initializing means 
    sigmas = [[[0 for k in range(number_of_parameters)] for j in range(number_of_parameters)] for i in range(10)] # initializing variances
    occurences = [0 for i in range(10)]
    pi = [0 for i in range(10)]
        
    """
    Learn mu
    """
    # Find Sums and Occurences
    for image in training_data:
        label = int(image[-1])
        occurences[label] += 1
        mu[label] = np.add(mu[label],image[:-1])
    # Calculate Averages and Prior Probabilities 
    for label,sums in enumerate(mu):
        mu[label] = np.multiply(1/(occurences[label] - 1), sums).reshape(-1,1) #unbaised estimator
        pi[label] = occurences[label] / number_of_total_samples
    """
    Learn sigma^2
    """
    time = 0
    for image in training_data:
        time += 1
        if time%10 == 0:
            progress(time,number_of_training_samples,suffix="training k= "+str(i))
        label = int(image[-1])
        image = image[:-1].reshape(-1,1)
        difference = np.subtract(image,mu[label])
        sigma = difference.dot(difference.T)
        sigmas[label] = np.add(sigmas[label],sigma)
    for label,sigma in enumerate(sigmas):
        sigmas[label] = np.multiply(1/(occurences[label]-1),sigma)
    sys.stdout.flush()
    """
    Method to find the best discriminant score
    """
    def estimateBestLabel(image):
        scores = [0 for _ in range(10)]
        # find score for each label 

        for label in range(10):
            inverted_variance = np.linalg.inv(sigmas[label])
            first_term = -0.5*image.T.dot(inverted_variance).dot(image)
            second_term = image.T.dot(inverted_variance).dot(mu[label])
            third_term = -0.5*mu[label].T.dot(inverted_variance).dot(mu[label])
            (sign, logdet) = np.linalg.slogdet(sigmas[label])
            fourth_term = -0.5*sign*logdet
            fifth_term = math.log(pi[label])
            score = first_term + second_term + third_term + fourth_term + fifth_term
            scores[label] = score[0][0]
        return(scores.index(max(scores)))
    correct = 0
    print("")
    for index, image in enumerate(testing_data):
        if index % 9 == 0:
            progress(index+1,1000,suffix="testing k= "+str(i))
        image = image[:-1].reshape(-1,1)
        if testing_data[index][-1] == estimateBestLabel(image):
            correct += 1
    sys.stdout.flush()
    print(correct/1000)
    final_estimates.append(correct/1000)
print("")
print(final_estimates)

[============================================================] 100.0% ...training k= 1
0.466========================================================] 100.0% ...testing k= 1
[============================================================] 100.0% ...training k= 2
0.498========================================================] 100.0% ...testing k= 2
[============================================================] 100.0% ...training k= 3
0.479========================================================] 100.0% ...testing k= 3
[============================================================] 100.0% ...training k= 4
0.473========================================================] 100.0% ...testing k= 4
[============================================================] 100.0% ...training k= 5
0.464========================================================] 100.0% ...testing k= 5
[============================================================] 100.0% ...training k= 6
0.49============================================

## Confusion Matrix for best K (k = 6)

In [40]:
data = np.load('dim_reducedImageData.npy')
data = np.concatenate((data,labels),axis=1)
number_of_parameters = len(data[0]) - 1 #209
number_of_total_samples = len(data)

k = 10
i = 6
training_data = np.concatenate((data[0:(i-1)*int(number_of_total_samples/k)],data[i*int(number_of_total_samples/k):number_of_total_samples]),axis=0)
testing_data = data[(i-1)*int(number_of_total_samples/k):i*int(number_of_total_samples/k)]
number_of_training_samples = len(training_data)
"""
Define Parameters 
"""
mu = [[0 for j in range(number_of_parameters)] for i in range(10)] # initializing means 
sigmas = [[[0 for k in range(number_of_parameters)] for j in range(number_of_parameters)] for i in range(10)] # initializing variances
occurences = [0 for i in range(10)]
pi = [0 for i in range(10)]

"""
Learn mu
"""
# Find Sums and Occurences
for image in training_data:
    label = int(image[-1])
    occurences[label] += 1
    mu[label] = np.add(mu[label],image[:-1])
# Calculate Averages and Prior Probabilities 
for label,sums in enumerate(mu):
    mu[label] = np.multiply(1/(occurences[label] - 1), sums).reshape(-1,1) #unbaised estimator
    pi[label] = occurences[label] / number_of_total_samples
"""
Learn sigma^2
"""
time = 0
for image in training_data:
    time += 1
    if time%10 == 0:
        progress(time,number_of_training_samples,suffix="training k= "+str(i))
    label = int(image[-1])
    image = image[:-1].reshape(-1,1)
    difference = np.subtract(image,mu[label])
    sigma = difference.dot(difference.T)
    sigmas[label] = np.add(sigmas[label],sigma)
for label,sigma in enumerate(sigmas):
    sigmas[label] = np.multiply(1/(occurences[label]-1),sigma)
sys.stdout.flush()
"""
Method to find the best discriminant score
"""
def estimateBestLabel(image):
    scores = [0 for _ in range(10)]
    # find score for each label 

    for label in range(10):
        inverted_variance = np.linalg.inv(sigmas[label])
        first_term = -0.5*image.T.dot(inverted_variance).dot(image)
        second_term = image.T.dot(inverted_variance).dot(mu[label])
        third_term = -0.5*mu[label].T.dot(inverted_variance).dot(mu[label])
        (sign, logdet) = np.linalg.slogdet(sigmas[label])
        fourth_term = -0.5*sign*logdet
        fifth_term = math.log(pi[label])
        score = first_term + second_term + third_term + fourth_term + fifth_term
        scores[label] = score[0][0]
    return(scores.index(max(scores)))
correct = 0
print("")
estimated = []
real = []
for index, image in enumerate(testing_data):
    if index % 9 == 0:
        progress(index+1,1000,suffix="testing k= "+str(i))
    image = image[:-1].reshape(-1,1)
    real_label = testing_data[index][-1]
    estimated_label = estimateBestLabel(image)
    real.append(real_label)
    estimated.append(estimated_label)
sys.stdout.flush()
confusion = confusion_matrix(real, estimated,labels=[0,1,2,3,4,5,6,7,8,9])

[============================================================] 100.0% ...training k= 6


In [41]:
"""
Print Confusion Matrix 
"""
image_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship' ,'truck']
confusion = pd.DataFrame(confusion)
confusion.insert(0, "", image_labels, True)
confusion.columns = ["", 'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship' ,'truck']
print(confusion.to_string(index=False))

             airplane  automobile  bird  cat  deer  dog  frog  horse  ship  truck
   airplane        46          12     2    5     3    2     2      1    22     10
 automobile         3          67     1    1     1    1     0      1     5     11
       bird        11           3    28   10    19    6     7      2     4     10
        cat         2           5     9   27     9   14     9      6     1     12
       deer         6           7     2    6    47    4     4      8     4      4
        dog         0           8     7   14     4   44     2      8     2      6
       frog         2           9     8   10     8    5    39      6     2      8
      horse         1           3     3    9     2    8     4     69     1     11
       ship         9          13     2    3     5    2     4      0    48     14
      truck         3          19     1    3     1    3     1      3     6     75


## (Model 2) Logistic Regression: Jon

## (Model 3) Support Vector Machine: Joe

## Results 

### QDA
[0.466, 0.498, 0.479, 0.473, 0.464, 0.49, 0.441, 0.469, 0.482, 0.473]

### Logistics Regression 

### SVM

--------------------------------------------------------------------------------------------------------------------------------------------

# Twitter Data - Classification 

## Preprocessing Step

Our twitter data preprocessing step consistions of the following: 
   1. Tokenizing words the tweets 
   2. Removing any links, @ mentions, and #tags that don't have semantic meaning outside of twitter
   3. Lemmatization: remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma
   4. Lower case all words 
   5. Remove stopwords: a commonly used word (such as “the”, “a”, “an”, “in”) that have little semantic meaning 

In [29]:
"""
Reading in Data
"""
df_raw_twitter_data = pd.read_csv('train.csv', header=None, encoding = "ISO-8859-1").values[1:]
df_raw_twitter_data = df_raw_twitter_data[:,1:] # getting rid of index 

labels = df_raw_twitter_data[:,0]

In [52]:
"""
Preprocessing
"""
print(np.shape(df_raw_twitter_data))
corpus = []
print("")
time = 0
for index,[sentiment, tweet] in enumerate(df_raw_twitter_data):
    time += 1
    # Tokenize Words 
    tokens = tweet.split(" ")
    # Remove Links, @ mentions, # tags
    links = ['http', '.com', '#', '@', '&', '~']
    tokens = [w for w in tokens if not any(x in w for x in links)]
    # Tokenize again 
    regex = re.compile('[^a-zA-Z]')
    tokens = [regex.sub('', w) for w in tokens]
    # Lemmatization 
    lemmatizer = WordNetLemmatizer() 
    tokens = [lemmatizer.lemmatize(w) for w in tokens ]
    # Clean Up
    tokens = [w.lower() for w in tokens if len(w) > 0]
    # Remove Stopwords
    stop_words = set(stopwords.words('english'))
    stop_words.add('im')
    tokens = [w for w in tokens if not w in stop_words]
    corpus.append(" ".join(tokens))
    if time % 100 == 0:
        progress(time,99988,suffix="running twitter processing")
sys.stdout.flush()


(99989, 2)



### Vectorizing the Words 
CountVectorizer() is a sklearn library that turns a list of strings into a $n\mathbb{x}p$ array. Each row corresponds to a sample $x_{i}$ and each column corresponds to a unique word. The value is the occurence of the word in the tweet. We are using a bag-of-words approach, the simplest approach.  

In [53]:
##############
star = 5000
##############
corpus = np.load('corpus.npy')
vectorizer = CountVectorizer()
vobj = vectorizer.fit_transform(corpus[:star])
vectors = vobj.toarray()
print("\nFinished vectorization")
print(np.shape(vectors))
np.save('corpus.npy', vectors)


Finished vectorization
(5000, 7882)


### PCA

In [8]:
# First we create a StandardScaler object to 0 mean the data matrix but preserve the variance
stand_scaler = sklpp.StandardScaler(with_mean = True, with_std = False)
# Fits the data matrix to the StandardScaler object defined ^
centered_ImageData = stand_scaler.fit_transform(vectors)

In [9]:
# Creates a PCA object that reduces the dimensions of our data matrix keeping 95% of the variance
pca_obj = skldecomp.PCA(n_components = 0.90, svd_solver = 'auto')
dim_reducedImageData = pca_obj.fit_transform(centered_ImageData)
np.save('twitterDataReduced.npy', dim_reducedImageData)

In [21]:
print('Data has been reduced to {} features after PCA'.format(dim_reducedImageData.shape[1]))

Data has been reduced to 1719 features after PCA


## (Model 1) Quadratic Discriminant Analysis: Justin

### QDA Implementation ###
With this QDA implementation I chose to go with sklearn. Sklearn's behind the scenes optimizations are simply 100x better than mine. Literally, it took around 43 minutes on average to train on each k with n=5000 and p=1719. I probably spent around 14+ hours playing around with optimizing my code and finally just tried sklearn's qda implentation. It finished in less than 5 minutes. 

In [58]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
"""
Pull pca data from saved
"""
data = np.load('twitterDataReduced.npy')
labels = labels[:5000]
number_of_parameters = len(data[0]) - 1 #5000
number_of_total_samples = len(data)
number_of_labels = 2
"""
K-Fold cross validation 
"""     
final_estimates = []
k = 10
for i in range(1,11):
    training_data = np.concatenate((data[0:(i-1)*int(number_of_total_samples/k)],data[i*int(number_of_total_samples/k):number_of_total_samples]),axis=0)
    training_labels = np.concatenate((labels[0:(i-1)*int(number_of_total_samples/k)],labels[i*int(number_of_total_samples/k):number_of_total_samples]),axis=0)
    testing_data = data[(i-1)*int(number_of_total_samples/k):i*int(number_of_total_samples/k)]
    number_of_training_samples = len(training_data)
    clf = QDA()
    print("Training on k="+str(i))
    clf.fit(training_data, training_labels)
    print("")
    correct = 0
    for index, image in enumerate(testing_data):
        if index % 1 == 0:
            progress(index+1,int(star/10),suffix="testing k= "+str(i))
        if int(testing_data[index][-1]) == int(clf.predict([image])[0]):
            correct += 1
    sys.stdout.flush()
    print("")
    final_estimates.append(correct/int(star/10))
print(final_estimates)


Training on k=1

[============================================================] 100.0% ...testing k= 1
Training on k=2

[============================================================] 100.0% ...testing k= 2
Training on k=3

[============================================================] 100.0% ...testing k= 3
Training on k=4

[============================================================] 100.0% ...testing k= 4
Training on k=5

[============================================================] 100.0% ...testing k= 5
Training on k=6

[============================================================] 100.0% ...testing k= 6
Training on k=7

[============================================================] 100.0% ...testing k= 7
Training on k=8

[============================================================] 100.0% ...testing k= 8
Training on k=9

[============================================================] 100.0% ...testing k= 9
Training on k=10

[======================================================

## (Model 2) Logistic Regression: Jon

## (Model 3) ??? : Joe

# Human Activity Clustering

### Preprocessing

In [93]:
"""
Opening File, creating nparray 
Data is already 0 mean 1 variance 
"""
features = open('human_activity_features_train_data.txt','r')
human_activity_data = []
for feature in features:
    feature = np.array([float(w) for w in features.readline().split(" ") if len(w) > 0])
    human_activity_data.append(feature)
features.close()
human_activity_data = np.array(human_activity_data)
print("features: ",np.shape(human_activity_data))

labels = open('human_activity_labels_train_data.txt','r')
human_activity_labels = []
for label in labels:
    human_activity_labels.append(int(label))
labels.close()
human_activity_labels = human_activity_labels[:3676]
print("labels: ",np.shape(human_activity_labels))

features:  (3676, 561)
labels:  (3676,)


### PCA

In [94]:
# First we create a StandardScaler object to 0 mean the data matrix but preserve the variance
stand_scaler = sklpp.StandardScaler(with_mean = True, with_std = False)
# Fits the data matrix to the StandardScaler object defined ^
centered_ImageData = stand_scaler.fit_transform(human_activity_data)

In [101]:
# Creates a PCA object that reduces the dimensions of our data matrix keeping 95% of the variance
pca_obj = skldecomp.PCA(n_components = 0.95, svd_solver = 'auto')
dim_reducedImageData = pca_obj.fit_transform(centered_ImageData)
np.save('human_activity_data.npy', dim_reducedImageData)

In [96]:
print('Data has been reduced to {} features after PCA'.format(dim_reducedImageData.shape[1]))

Data has been reduced to 153 features after PCA


### (Model 1) Hierarchical/Agglomerative Clustering - Justin

This type of clustering breaks all data points down into centroids and groups them one by one until it reaches the specified number of clusters, k. The linkage policy determines grouping, which are:
 - simple: closest distance between clusters 
 - complete: farthest distance between clusters 
 - average: average distance between clusters 
 - ward: sum of squared differences
Our implementation is using euclidean distance 

Hierarchical/Agglomerative is deterministic and⁠—as compared to k-means⁠—is slow. Complete, Average, and Ward linkage policies yield a $n^{3}$ runtime. Simple linkage yields $n^{2}$ runtime with clever optimizations, which is why we are using sklearn. 

We are using k=6 because of our a-priori knoweldge that there are 6 groups:
 1. WALKING,
 2. WALKING_UPSTAIRS,
 3. WALKING_DOWNSTAIRS,
 4. SITTING,
 5. STANDING,
 6. LAYING;


In [102]:
human_activity_data = np.load('human_activity_data.npy')
clustering = AgglomerativeClustering(n_clusters = 6, linkage='single').fit(human_activity_data)
predicted_clusters = clustering.labels_
predicted_results = [[0 for j in range(6)] for i in range(6)]
for index,cluster in enumerate(predicted_clusters):
    predicted_results[cluster][human_activity_labels[index]-1] += 1   
for i in range(6):
    Sum = sum(predicted_results[i])
    predicted_results[i] = [(x/Sum)*100 for x in predicted_results[i]]
    print("Cluster ",i,"best predicted label is ",predicted_results[i].index(max(predicted_results[i])),"with ",'%.3f'%(max(predicted_results[i])),"% with cluster size", Sum," account for ",'%.3f'%(Sum/3676*100), "% of the data")

Cluster  0 best predicted label is  4 with  18.801 % with cluster size 3670  account for  99.837 % of the data
Cluster  1 best predicted label is  5 with  100.000 % with cluster size 1  account for  0.027 % of the data
Cluster  2 best predicted label is  3 with  50.000 % with cluster size 2  account for  0.054 % of the data
Cluster  3 best predicted label is  5 with  100.000 % with cluster size 1  account for  0.027 % of the data
Cluster  4 best predicted label is  5 with  100.000 % with cluster size 1  account for  0.027 % of the data
Cluster  5 best predicted label is  5 with  100.000 % with cluster size 1  account for  0.027 % of the data


In [103]:
human_activity_data = np.load('human_activity_data.npy')
clustering = AgglomerativeClustering(n_clusters = 6, linkage='complete').fit(human_activity_data)
predicted_clusters = clustering.labels_
predicted_results = [[0 for j in range(6)] for i in range(6)]
for index,cluster in enumerate(predicted_clusters):
    predicted_results[cluster][human_activity_labels[index]-1] += 1   
for i in range(6):
    Sum = sum(predicted_results[i])
    predicted_results[i] = [(x/Sum)*100 for x in predicted_results[i]]
    print("Cluster ",i,"best predicted label is ",predicted_results[i].index(max(predicted_results[i])),"with ",'%.3f'%(max(predicted_results[i])),"% with cluster size", Sum," account for ",'%.3f'%(Sum/3676*100), "% of the data")

Cluster  0 best predicted label is  2 with  28.571 % with cluster size 63  account for  1.714 % of the data
Cluster  1 best predicted label is  0 with  27.500 % with cluster size 360  account for  9.793 % of the data
Cluster  2 best predicted label is  5 with  23.553 % with cluster size 2021  account for  54.978 % of the data
Cluster  3 best predicted label is  5 with  100.000 % with cluster size 9  account for  0.245 % of the data
Cluster  4 best predicted label is  3 with  35.294 % with cluster size 17  account for  0.462 % of the data
Cluster  5 best predicted label is  3 with  23.466 % with cluster size 1206  account for  32.807 % of the data


In [104]:
human_activity_data = np.load('human_activity_data.npy')
clustering = AgglomerativeClustering(n_clusters = 6, linkage='average').fit(human_activity_data)
predicted_clusters = clustering.labels_
predicted_results = [[0 for j in range(6)] for i in range(6)]
for index,cluster in enumerate(predicted_clusters):
    predicted_results[cluster][human_activity_labels[index]-1] += 1   
for i in range(6):
    Sum = sum(predicted_results[i])
    predicted_results[i] = [(x/Sum)*100 for x in predicted_results[i]]
    print("Cluster ",i,"best predicted label is ",predicted_results[i].index(max(predicted_results[i])),"with ",'%.3f'%(max(predicted_results[i])),"% with cluster size", Sum," account for ",'%.3f'%(Sum/3676*100), "% of the data")

Cluster  0 best predicted label is  5 with  100.000 % with cluster size 2  account for  0.054 % of the data
Cluster  1 best predicted label is  0 with  41.463 % with cluster size 41  account for  1.115 % of the data
Cluster  2 best predicted label is  3 with  22.640 % with cluster size 1568  account for  42.655 % of the data
Cluster  3 best predicted label is  2 with  37.037 % with cluster size 27  account for  0.734 % of the data
Cluster  4 best predicted label is  5 with  38.462 % with cluster size 13  account for  0.354 % of the data
Cluster  5 best predicted label is  5 with  23.506 % with cluster size 2025  account for  55.087 % of the data


In [105]:
human_activity_data = np.load('human_activity_data.npy')
clustering = AgglomerativeClustering(n_clusters = 6, linkage='ward').fit(human_activity_data)
predicted_clusters = clustering.labels_
predicted_results = [[0 for j in range(6)] for i in range(6)]
for index,cluster in enumerate(predicted_clusters):
    predicted_results[cluster][human_activity_labels[index]-1] += 1   
for i in range(6):
    Sum = sum(predicted_results[i])
    predicted_results[i] = [(x/Sum)*100 for x in predicted_results[i]]
    print("Cluster ",i,"best predicted label is ",predicted_results[i].index(max(predicted_results[i])),"with ",'%.3f'%(max(predicted_results[i])),"% with cluster size", Sum," account for ",'%.3f'%(Sum/3676*100), "% of the data")

Cluster  0 best predicted label is  5 with  26.484 % with cluster size 657  account for  17.873 % of the data
Cluster  1 best predicted label is  0 with  25.472 % with cluster size 742  account for  20.185 % of the data
Cluster  2 best predicted label is  3 with  24.784 % with cluster size 811  account for  22.062 % of the data
Cluster  3 best predicted label is  0 with  26.875 % with cluster size 640  account for  17.410 % of the data
Cluster  4 best predicted label is  5 with  25.371 % with cluster size 741  account for  20.158 % of the data
Cluster  5 best predicted label is  2 with  23.529 % with cluster size 85  account for  2.312 % of the data


## (Model 2) ?? - Jon

## (Model 3) K-Nearnest Neighbors (KNN) - Joe